In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import os
#data_dir = "sam_data/sa_000020/image_dir"
data_dir = "/home/ubuntu/MySAM/sam_data/mini_image_dir/image_dir"#fix me
image_path_iter = DataLoader(os.listdir(data_dir), batch_size=2, shuffle=True)
#image_paths = 
#image_iter = DataLoader(ImageFolder(data_dir, transform=transforms.ToTensor()), batch_size=2, shuffle=True) 

In [2]:
import cv2
def load_image(image_path, root_dir):
    #image_path = "/home/ubuntu/sam_data/sa_000020/mini_image_dir/train/sam:1.jpg"
    image = cv2.imread(''.join([root_dir, '/', image_path]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return torch.from_numpy(image).permute(2, 0, 1)
    

In [3]:
paths = data_dir.split('/')
paths[-1] = 'embedding_dir'
embedding_dir = "/".join(paths)
if not os.path.exists(embedding_dir):
    os.mkdir(embedding_dir)

In [5]:
import torch
with torch.no_grad():
    from build_sam import sam_model_registry
    teacher_model_checkpoint_path = 'model_checkpoint/sam_vit_h_4b8939.pth'
    teacher_model = sam_model_registry['default'](teacher_model_checkpoint_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    _ = teacher_model.to(device)
    for batch_path in image_path_iter:
        input_images = torch.stack([teacher_model.preprocess(load_image(path, data_dir).to(device)) for path in batch_path])
        #input_images = torch.stack([teacher_model.preprocess(image[0].to(device)) for image in batch_image])
        #pdb.set_trace()
        teacher_emd = teacher_model.image_encoder(input_images)
        for image_file_name, embed in zip(batch_path, teacher_emd):
            target_path = ''.join([embedding_dir, '/', image_file_name.split('.')[0], '.pth'])
            torch.save(embed, target_path)
        print("process one batch")
    

process one batch


In [1]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

1.7.0
0.8.0
